In [1]:
import logging
from copy import deepcopy
import pickle

import numpy as np
import pandas as pd
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import MultilayerPerceptronClassifier
from regression import CustomRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# conf = SparkConf()
# spark = SparkContext.getOrCreate(conf=conf)

In [15]:
train = pd.read_pickle('train_df.p')
test = pd.read_pickle('test_df.p')

# keys = set(train.keys())
# keys.remove('000001.SS')
# keys.remove('Target')
# ass = VectorAssembler(inputCols=list(keys), outputCol="features")
# df = spark.createDataFrame(train)
# output = ass.transform(df)
# test_df = spark.createDataFrame(test)
# test_fe = ass.transform(test_df)

In [9]:
lr = LinearRegression(featuresCol="features", labelCol='Target', maxIter=10000)
model = lr.fit(output)
test_re = model.transform(test_fe)

In [6]:
algorithm = CustomRegression(MLPRegressor(hidden_layer_sizes=(16,)))
model = algorithm.fit(np.array(output.select('features').rdd.map(lambda x: np.array(list(x)[0])).collect()), 
                      output.select('Target').rdd.map(lambda x: list(x)[0]))

In [7]:
# data_df = test_fe.select('features').toPandas()
model.predict(test_fe.select('features').rdd.map(lambda x: np.array(list(x)[0])).collect()).collect()

Help on class CustomRegression in module regression.algorithms:

class CustomRegression(MassRegressionAlgorithm)
 |  Custom mass univariate regression algorithm.
 |  
 |  Parameters
 |  ----------
 |  
 |  algorithm : object
 |      An object encapsulating a regression algorithm.
 |  
 |      Can be a scikit-learn algorithm or a custom algorithm that adheres to a
 |      scikit-learn style API. Must implement a fit(X, y) method that takes a
 |      deisgn matrix and response vector (X and y, both NumPy ndarrays) and
 |      returns a fitted model. The fitted model should expose data members
 |      coef_ and intercept_ that contain the fitted coefficients and intercept
 |      (if any). The fitted model should also implement a score(X, y) method
 |      giving the r-squared of fit for the given data and a predict(X) method
 |      that predicts responses given a design matrix.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, algorithm)
 |  
 |  -----------------------------------

In [18]:
alg = MLPRegressor(hidden_layer_sizes=(16,))


In [30]:
from sklearn.datasets import make_regression
X, Y = make_regression(n_samples=100, n_features=3, n_informative=3, n_targets=1, noise=1.0)
algorithm = CustomRegression(MLPRegressor(hidden_layer_sizes=(16,)))
model = algorithm.fit(X, Y.T)

In [20]:
Y = output.select('Target').rdd.map(lambda x: list(x)[0])
X = np.array(output.select('features').rdd.map(lambda x: np.array(list(x)[0])).collect())
from regression.utils import toseries
y = toseries(Y)

In [ ]:
output.select('Target').rdd.map(lambda x: list(x)[0]).collect()

In [34]:
toseries(Y).map(lambda v: deepcopy(alg).fit(X, v)).first()

array([ MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(16,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)], dtype=object)

In [2]:
df = spark.read.load('/Users/warn/PycharmProjects/Dissertation/src/stockforecaster/test.parquet')

In [13]:
input_num = 26
layers = [input_num, input_num / 3 * 2, input_num / 3, 2]
mlpc = MultilayerPerceptronClassifier(featuresCol="features", labelCol='ChangeDirection',
                                      layers=layers,
                                      predictionCol='DirPrediction')
model = mlpc.fit(df)

In [15]:
# pdf = df.toPandas()
# pdf['features'].values
spark.stop()

In [4]:
# import numpy as np
pnparray = pdf['features'].values
container = np.zeros((pnparray.shape[0], len(pnparray[0])))
for i in range(pnparray.shape[0]):
    container[i, :] = pnparray[i][:]

In [2]:
with open('/Users/warn/PycharmProjects/Dissertation/src/stockforecaster/ann_result.p') as f:
    data = pickle.load(f)
    
df = spark.read.load('/Users/warn/PycharmProjects/Dissertation/src/stockforecaster/test_df.parquet')

In [3]:
pdf = pd.DataFrame(data, columns=['prediction'])
sdf = spark.createDataFrame(pdf)
# df.withColumn('prediction', getattr(sdf, 'prediction'))

In [6]:
df.join(sdf).collect()

[Row(0=0.4283254687516453, 1=0.10087571640521305, 2=-0.7436187291337362, 3=-0.6966913607470953, 4=-0.7172825110220657, 5=0.11631991626398763, 6=-0.34889714318177656, 7=0.3655793217039771, 8=0.1668133299306388, 9=0.70922318511313, 10=-0.963570798567138, 11=-0.19470363111805636, 12=-1.7370772808398953, 13=2.1035152305092715, 14=1.096664352283045, 15=0.9112610641874274, 16=-0.928553146655458, 17=-0.2658651055724139, 18=-0.37983993447733133, 19=-0.15813646622622915, 20=-0.06894198304408679, 21=-0.5438936345997287, 22=-0.5263837729482536, 23=-0.4520554519239574, 24=-0.006515707529671733, 25=0.5241347046647357, Target=9.4, TodayPrice=9.44, features=DenseVector([0.4283, 0.1009, -0.9636, -0.1947, -1.7371, 2.1035, 1.0967, 0.9113, -0.9286, -0.2659, -0.3798, -0.1581, -0.7436, -0.0689, -0.5439, -0.5264, -0.4521, -0.0065, 0.5241, -0.6967, -0.7173, 0.1163, -0.3489, 0.3656, 0.1668, 0.7092]), prediction=0.07500062137842178),
 Row(0=0.43600100872110104, 1=0.027950424566955553, 2=-0.8037336213458977, 3=

DataFrame[prediction: double, test: double]